In [1]:
from Bio.SeqIO import FastaIO
from tqdm import tqdm

import sys
sys.path.insert(0, "../../")
from seqlearn.seq2vec.gensim import Seq2Vec

In [2]:
seq2vec = Seq2Vec(vec_size=100, k=6, stride=1)

In [3]:
neg_fasta = "../../data/Classification/ABI5-neg.fasta"
pos_fasta = "../../data/Classification/ABI5-pos.fasta"

neg_seqs = []; pos_seqs = []
for fasta in neg_fasta, pos_fasta:
    with open(fasta) as f:
        for s in tqdm(FastaIO.FastaIterator(f)):
            seq = str(s.seq).upper()
            if seq2vec.vocab.unknow_char in seq:
                continue
            if fasta == neg_fasta:
                neg_seqs.append(seq)
            else:
                pos_seqs.append(seq)

2987it [00:00, 128394.87it/s]
2987it [00:00, 87511.95it/s]


In [4]:
seq2vec.build_from_seqs(neg_seqs+pos_seqs)

5960it [00:41, 143.93it/s]


In [5]:
neg_vecs = []; pos_vecs = []
for seqs in neg_seqs, pos_seqs:
    for seq in tqdm(seqs):
        try:
            v = seq2vec.seq_to_vec(seq)
        except AssertionError:
            continue
        
        if seqs == neg_seqs:
            neg_vecs.append(v)
        else:
            pos_vecs.append(v)

100%|██████████| 2987/2987 [00:28<00:00, 106.05it/s]


In [8]:
import numpy as np

X_neg = np.vstack(neg_vecs)
X_pos = np.vstack(pos_vecs)
X = np.vstack([X_neg, X_pos])
y = np.hstack([np.zeros(len(neg_vecs)), np.ones(len(pos_vecs))])

In [11]:
X.shape, y.shape

((5959, 100), (5959,))